# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import models,layers
from tensorflow.keras import regularizers


In [2]:
df = pd.read_csv('./data/creditcard.csv')
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)
# data["Class"].value_counts()
# x_data= data.drop(columns="Class", axis=0)
# label = data["Class"]

In [4]:
# min_d = data.min()
# max_d = data.max()
# normalized_df=(data - min_d)/(max_d - min_d)
# normalized_df.head()

In [5]:
train_data = data.sample(frac= 0.7, random_state=125)
test_data =  data.drop(train_data.index)

In [6]:
train_labels = train_data.iloc[:,-1]
train_data = train_data.iloc[:,0:-1]
test_labels = test_data.iloc[:,-1]
test_data = test_data.iloc[:,0:-1]

In [7]:
train_mean = train_data.mean(axis=0) # taking the mean of 
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [8]:
train_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,...,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03
mean,-1.109417e-16,1.290292e-16,1.825664e-16,-6.693565e-16,-3.766540e-17,3.269438e-16,-9.748693e-18,-3.135695e-16,1.562510e-17,-2.495182e-16,...,-3.802796e-17,-5.134933e-17,-2.779586e-17,-3.359673e-17,-1.869171e-17,5.530973e-17,2.610394e-17,-9.063867e-19,-4.125067e-17,7.396116e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.862678e+00,-6.591548e+00,-6.090114e+00,-5.873685e+00,-2.216012e+00,-6.350348e+00,-5.388087e+00,-9.017128e+00,-1.068333e+01,-6.500345e+00,...,-7.224868e+00,-1.049826e+01,-8.931589e+00,-1.982055e+01,-3.530342e+00,-8.106762e+00,-3.019090e+00,-9.162196e+00,-6.780456e+00,-4.493455e-01
25%,-8.594385e-01,-6.094010e-02,-4.528364e-01,-2.241311e-02,-6.361540e-01,-8.170912e-02,-4.343787e-01,6.583658e-02,-7.885326e-02,-2.865213e-01,...,-2.929234e-01,-1.604649e-01,-5.496662e-01,-2.002724e-01,-5.699826e-01,-5.563201e-01,-6.627150e-01,-1.182502e-01,-2.023182e-01,-4.318147e-01
50%,-1.998142e-01,1.942204e-01,-1.879850e-01,2.923051e-01,-2.709968e-01,1.842961e-01,-5.201991e-02,2.832341e-01,-4.695194e-03,1.938894e-01,...,-1.037443e-01,-5.420475e-02,-1.171806e-02,-2.329787e-02,7.711142e-02,4.016678e-02,-1.038414e-01,-1.794239e-02,9.130863e-03,-3.406208e-01
75%,9.793281e-01,5.636379e-01,1.318854e-01,5.240119e-01,3.298107e-01,4.102007e-01,3.879573e-01,4.138186e-01,1.144849e-01,5.395002e-01,...,1.671247e-01,9.768494e-02,5.190013e-01,1.622977e-01,7.534750e-01,6.158174e-01,5.654196e-01,2.321295e-01,3.104171e-01,-3.398456e-03
max,1.718662e+00,8.343872e-01,6.975561e+00,1.027207e+00,4.039376e+00,4.810666e+00,4.699684e+00,1.661383e+00,5.170253e+00,2.492623e+00,...,1.123292e+01,1.238893e+01,8.351994e+00,7.559058e+00,6.012357e+00,3.708442e+00,6.387233e+00,4.792341e+00,8.734119e+00,1.254459e+01


In [9]:
#train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [10]:
train_data = train_data.to_numpy()
train_labels = train_labels.to_numpy().astype('float64')

test_data = test_data.to_numpy()
test_labels = test_labels.to_numpy().astype('float64')

In [11]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [13]:
model.fit(train_data, train_labels, epochs=50,validation_split = 0.3)

Epoch 1/50
31/31 [==============================] - 5s 58ms/step - loss: 0.9144 - accuracy: 0.7232 - val_loss: 0.6966 - val_accuracy: 0.7295
Epoch 2/50
31/31 [==============================] - 0s 8ms/step - loss: 0.8455 - accuracy: 0.6952 - val_loss: 0.6680 - val_accuracy: 0.7295
Epoch 3/50
31/31 [==============================] - 0s 11ms/step - loss: 0.8944 - accuracy: 0.7119 - val_loss: 0.6451 - val_accuracy: 0.7295
Epoch 4/50
31/31 [==============================] - 0s 9ms/step - loss: 0.8541 - accuracy: 0.7032 - val_loss: 0.6242 - val_accuracy: 0.7295
Epoch 5/50
31/31 [==============================] - 0s 11ms/step - loss: 0.6802 - accuracy: 0.7489 - val_loss: 0.6121 - val_accuracy: 0.7295
Epoch 6/50
31/31 [==============================] - 0s 11ms/step - loss: 0.7485 - accuracy: 0.6972 - val_loss: 0.5980 - val_accuracy: 0.7295
Epoch 7/50
31/31 [==============================] - 0s 9ms/step - loss: 0.6628 - accuracy: 0.7445 - val_loss: 0.5783 - val_accuracy: 0.7295
Epoch 8/50
31/31

In [14]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 3ms/step - loss: 0.3656 - accuracy: 0.9525


In [15]:
loss

0.365626722574234

In [16]:
accuracy

0.9525423645973206

In [17]:
predictions = model.predict(test_data)

In [18]:
y_pred = (predictions > 0.5)

In [19]:
con = tf.math.confusion_matrix(
    test_labels, y_pred, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None
)

In [20]:
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[448,   0],
       [ 28, 114]], dtype=int32)>

In [21]:
result = accuracy * 100
result

95.25423645973206